In [ ]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import (
    CFEfficacyAgent,
    ToxicityScreeningAgent,
    CompoundPrioritizationAgent,
)

In [ ]:
# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("Optimize Efficacy Tools")
mlflow.autolog()
mlflow.tracing.disable_notebook_display()

load_dotenv("../.env")
dspy.configure(
    lm=dspy.LM("gemini/gemini-2.5-pro", temperature=0.5, cache=False, rollout_id=1)
)

COMPOUND = "Luminespib"

In [ ]:
efficacy_agent = CFEfficacyAgent(max_iters=12)
# efficacy_agent.agent.react.signature = efficacy_agent.agent.react.signature.insert(
#     0,
#     "tool_expected_info",
#     dspy.OutputField(
#         desc="TRUE or FALSE: <more info>. If FALSE, describe what information (ex databases, literature, etc) was desired from tool. Do not say FALSE if tool did not return any information (ex due to no information available), only say FALSE if tool returned information that was not useful or other information was desired from this tool call.\n\n"
#     ),
#     type_=str,
# )
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

In [ ]:
# tox_agent = ToxicityScreeningAgent(max_iters=12)
# tox_results = tox_agent(compound_name=COMPOUND)
# print(tox_results)

In [ ]:
# prioritization_agent = CompoundPrioritizationAgent()
# prioritization_results = prioritization_agent(compound_name=COMPOUND)
# print(prioritization_results)